In [4]:
from judo_footage_analysis.utils import get_spark

root = "/cs-share/pradalier/tmp/judo"

spark = get_spark(cores=4, mem="1g")
labels = spark.read.json(
    f"{root}/data/combat_phase/discrete_v2/labels.json", multiLine=True
)
labels.printSchema()
labels.show(truncate=False, n=10)

root
 |-- file: string (nullable = true)
 |-- is_active: long (nullable = true)
 |-- is_match: long (nullable = true)
 |-- is_standing: long (nullable = true)
 |-- time: long (nullable = true)

+--------------------------+---------+--------+-----------+----+
|file                      |is_active|is_match|is_standing|time|
+--------------------------+---------+--------+-----------+----+
|data/clips/mat_01/0002.mp4|0        |1       |0          |0   |
|data/clips/mat_01/0002.mp4|0        |1       |0          |1   |
|data/clips/mat_01/0002.mp4|0        |1       |0          |2   |
|data/clips/mat_01/0002.mp4|0        |1       |0          |3   |
|data/clips/mat_01/0002.mp4|0        |1       |0          |4   |
|data/clips/mat_01/0002.mp4|0        |1       |0          |5   |
|data/clips/mat_01/0002.mp4|1        |1       |1          |6   |
|data/clips/mat_01/0002.mp4|1        |1       |1          |7   |
|data/clips/mat_01/0002.mp4|1        |1       |1          |8   |
|data/clips/mat_01/0002.mp

24/04/04 01:00:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
labels.select("file").distinct().count()

19

In [10]:
labels.drop("file", "time").describe().show()

+-------+------------------+-------------------+-------------------+
|summary|         is_active|           is_match|        is_standing|
+-------+------------------+-------------------+-------------------+
|  count|               570|                570|                570|
|   mean|0.4964912280701754| 0.7684210526315789|0.18596491228070175|
| stddev| 0.500426851990946|0.42221189040527146| 0.3894200981285407|
|    min|                 0|                  0|                  0|
|    max|                 1|                  1|                  1|
+-------+------------------+-------------------+-------------------+



In [15]:
cols = ["is_match", "is_active", "is_standing"]
labels.groupBy(*cols).count().orderBy(*cols, ascending=False).show()

+--------+---------+-----------+-----+
|is_match|is_active|is_standing|count|
+--------+---------+-----------+-----+
|       1|        1|          1|  106|
|       1|        1|          0|  177|
|       1|        0|          0|  155|
|       0|        0|          0|  132|
+--------+---------+-----------+-----+



In [20]:
from pyspark.sql import functions as F

labels.select(
    F.round(F.sum("is_match") / F.count("*"), 3).alias("pct_match"),
    F.round(F.sum("is_active") / F.sum("is_match"), 3).alias("pct_active"),
    F.round(F.sum("is_standing") / F.sum("is_match"), 3).alias("pct_standing"),
).show()

+---------+----------+------------+
|pct_match|pct_active|pct_standing|
+---------+----------+------------+
|    0.768|     0.646|       0.242|
+---------+----------+------------+

